## Workspaces Demo (OUTDATED)

NOTE: This notebook is outdated, as it has been superceded by the
individual loading and saving notebooks.

This notebook will demonstrate how to register a worksapce, save results of a process graph to a workspace
and load those results in a new process graph. The process graph will be executed on the EODC openEO backend. 

In [ ]:
import openeo

# Set these variables to your own values if you intent to skip step 0 so you can run this code!

WORKSPACE_NAME = '' # Corresponds to your bucket name in whatever S3 endpoint you are using

# Connect to the openEO backend and authenticate with EGI Check-In

connection = openeo.connect("https://openeo.eodc.eu/openeo/1.1.0")
connection = connection.authenticate_oidc(provider_id="egi")

## (optional) Step 0: Create a workspace @ EODC if you don't have your own.

For this we will send a post request to the /workspaces/{workspace_name} endpoint,
with the body containing the workspace configuration as seen in the example below.

#### IMPORTANT

if you have your own bucket/blob you do not need to create a workspace @ EODC,
instead you can skip to step 1 

In [ ]:
import requests

int_ws = {
    "preferred_name": WORKSPACE_NAME,
}


response = requests.post(
    url="https://openeo.eodc.eu/openeo/1.1.0/workspaces",
    json=int_ws,
    headers={
        "authorization": f"Bearer {connection.auth.bearer}"
    },
)

response.content

In [ ]:
import requests



response = requests.get(
    url="https://openeo.eodc.eu/openeo/1.1.0/workspaces/user",
    headers={
        "authorization": f"Bearer {connection.auth.bearer}"
    },
)

response.content

In [ ]:
# Enter your received credentials here for later!

S3_ENDPOINT = ""
S3_ACCESS_KEY = ""
S3_SECRET_KEY = ""

## Step 1: Register an external workspace with the EODC backend.

For this we will send a post request to the /workspaces/{workspace_name}/register endpoint,
with the body containing the workspace configuration as seen in the example below.

You can skip this step if you did step 0

In [ ]:
import requests

ext_ws = {
    "storage_type": "ceph",
    "credentials": {
        "credentials": {
            "access_key": S3_ACCESS_KEY,
            "secret_key": S3_SECRET_KEY,
        }
    },
    "url": S3_ENDPOINT,
}


response = requests.post(
    url=f"https://openeo.eodc.eu/openeo/1.1.0/workspaces/{WORKSPACE_NAME}/register",
    json=ext_ws,
    headers={
        "authorization": f"Bearer {connection.auth.bearer}"
    },
)



response

Now the endpoint is registered with the backend and we can use it to save and load results.

## Step 2: Save the results of a process graph to the workspace.

For this we will use the openeo python library and execute the following process graph which will load a small spatial extent of the boa_sentinel_2 dataset, and save it to the specified workspace.

In [ ]:
import uuid


process_graph = {
    "id": str(uuid.uuid4()),
    "process_graph": {
        "load1": {
            "process_id": "load_collection",
            "arguments": {
                "id": "boa_sentinel_2",
                "spatial_extent": {
                    "west": 16.156771491786476,
                    "east": 16.59018048465475,
                    "south": 48.08419286799747,
                    "north": 48.34670064966687,
                },
                "temporal_extent": [
                    "2019-01-01T00:00:00Z",
                    "2019-01-31T00:00:00Z",
                ],
                "bands": ["B02"],
            },
        },
        "save2": {
            "process_id": "save_result",
            "arguments": {
                "data": {"from_node": "load1"},
                "options": {"workspace": WORKSPACE_NAME},
                "format": "GTIFF",
            },
            "result": True,
        },
    },
}


job = connection.create_job(process_graph=process_graph, title="save-to-workspace-job")
job.start_job()

In [ ]:
job # Execute this cell to check on your started jobs status

## Step 3: Check the results in the workspace and get the resulting collection id

For this we can either use any external tool to check the results in the workspace, 
or use the eodc-sdk WorkspaceAdapter (or directly use the CephAdapter) to list the files in the workspace.



In [ ]:

from eodc.workspace import WorkspaceAdapter, StorageType # , CephAdapter 

adapter: WorkspaceAdapter = WorkspaceAdapter.create_adapter(storage_type=StorageType.CEPH, parameters={"url": S3_ENDPOINT, "access_key": S3_ACCESS_KEY, "secret_key": S3_SECRET_KEY})

# Equivalent CephAdapter initialization
# adapter = CephAdapter(S3_ENDPOINT, SecretStr(S3_ACCESS_KEY),  SecretStr(S3_SECRET_KEY))

adapter.list_workspace_files(WORKSPACE_NAME)



Here we can see that all our results are present and listed. The collection id corresponds to the created folder name.
The STAC collection data is found in the STAC folder, 
and the created files are in the first level of the folder, while the STAC items are found in the ../STAC/items/ 


## Step 4: Load the results of the process graph from the workspace and use it in a new process graph.

By adding the resulting collection id (the name of the resulting folder) for example: "openeo_abcd-1234-5678..."
as the id in the load collection call and the registered Workspace name as the "workspace" property, you can reload the saved data from the original job
and use it in any new process graph.

In [ ]:
process_graph = {
    "id": str(uuid.uuid4()),
    "process_graph": {
        "load1": {
            "process_id": "load_collection",
            "arguments": {
                "id": "openeo_75cbd3f2-6296-479d-9231-53ffca1254ad",
                "properties": {"workspace": WORKSPACE_NAME},
                "spatial_extent": {
                    "west": 15,
                    "east": 48,
                    "south": 17,
                    "north": 49       
                },
                "temporal_extent": [
                    "2019-01-01T00:00:00Z",
                    "2024-01-08T00:00:00Z",
                ],
                "bands": ["raster-result"],
            },
        },
        "save2": {
            "process_id": "save_result",
            "arguments": {"data": {"from_node": "load1"}, "format": "GTIFF"},
            "result": True,
        },
    },
}


job = connection.create_job(process_graph=process_graph, title="load-from-workspace-job")
job.start_job()

In [ ]:
job # Execute this cell to check on your started jobs status

## (optional) Step 5: Share your results with another user

You can easily give other users read-access to your (EODC)-Workspace by sending a post request
to the /workspace/{workspace_name}/share/{user_id}" endpoint, which will then give read-access to the given user.

In [ ]:
SHAREE_USER_ID = ""

import requests

response = requests.post(
    url=f"https://openeo.eodc.eu/openeo/1.1.0/workspace/{WORKSPACE_NAME}/share/{SHAREE_USER_ID}",
    headers={
        "authorization": f"Bearer {connection.auth.bearer}"
    },
)

response.content